In [1]:
from d2l import mxnet as d2l
from mxnet import autograd, gluon, np, npx
from mxnet.gluon import nn
import mxnet as mx
import random
import sys
npx.set_np()

In [3]:
from mxnet import np, npx
from mxnet.gluon import nn, Block

npx.set_np()

class NeuMF(nn.Block):
    def __init__(self, num_factors, num_users, num_items, nums_hiddens, **kwargs):
        super(NeuMF, self).__init__(**kwargs)

        # Embedding cho GMF
        self.P = nn.Embedding(num_users, num_factors)
        self.Q = nn.Embedding(num_items, num_factors)

        # Embedding cho MLP
        self.U = nn.Embedding(num_users, num_factors)
        self.V = nn.Embedding(num_items, num_factors)

        # MLP layers
        self.mlp = nn.Sequential()
        for num_hiddens in nums_hiddens:
            self.mlp.add(nn.Dense(num_hiddens, activation='relu', use_bias=True))

        # Lớp dự đoán cuối cùng
        self.prediction_layer = nn.Dense(1, activation='sigmoid', use_bias=False)

    def forward(self, user_id, item_id):
        # GMF: Nhân từng phần tử giữa hai embedding
        p_mf = self.P(user_id)
        q_mf = self.Q(item_id)
        gmf = p_mf * q_mf

        # MLP: Nối hai embedding và đưa vào MLP
        p_mlp = self.U(user_id)
        q_mlp = self.V(item_id)
        mlp_input = np.concatenate([p_mlp, q_mlp], axis=1)
        mlp_output = self.mlp(mlp_input)

        # Nối đầu ra của GMF và MLP
        concat = np.concatenate([gmf, mlp_output], axis=1)

        # Dự đoán xác suất
        return self.prediction_layer(concat)


In [4]:
import random
from mxnet.gluon import data

class PRDataset(data.Dataset):
    def __init__(self, users, items, candidates, num_items):
        self.users = users
        self.items = items
        self.cand = candidates
        self.all = set(range(num_items))  # Sử dụng range thay vì list cho hiệu suất tốt hơn

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        # Lấy các sản phẩm chưa được tương tác (sản phẩm âm)
        neg_items = list(self.all - set(self.cand[int(self.users[idx])]))
        # Chọn ngẫu nhiên một sản phẩm âm từ neg_items
        neg_item = random.choice(neg_items)
        return self.users[idx], self.items[idx], neg_item
